# Helper Functions

In [1]:
import numpy as np
import os
from PIL import Image
import Sokoban
import random

def numpyToString(array):
    gameCharacters="# @$.+*"
    char_to_int = dict((c, i) for i, c in enumerate(gameCharacters))
    int_to_char = dict((i, c) for i, c in enumerate(gameCharacters))
    intArray = np.argmax(array, axis=0)
    output=""
    for (i,j), index in np.ndenumerate(intArray):
        if i > 0 and j == 0:
            output += "\n"
        output += int_to_char[index]
    return output

def stringToNumpy(string):
    data = np.array([list(l) for l in string.split("\n")])
    
    gameCharacters="# @$.+*"
    char_to_int = dict((c, i) for i, c in enumerate(gameCharacters))
    int_to_char = dict((i, c) for i, c in enumerate(gameCharacters))

    encodingData = np.zeros((len(gameCharacters), data.shape[0], data.shape[1]))
    for (i,j),c in np.ndenumerate(data):
        index=char_to_int[c]
        encodingData[index][i][j]=1
    return encodingData

def numpyToImage(array, tileSize=16):
    width = array.shape[2]
    height = array.shape[1]
    lvlRows = numpyToString(array).split("\n")
    graphics = {
        "@":"assets/player.png",
        "#":"assets/wall.png",
        "$":"assets/box.png",
        "*":"assets/box_target.png",
        ".":"assets/target.png",
        "+":"assets/player_target.png"
    }
    for c in graphics:
        graphics[c] = Image.open(graphics[c]).convert('RGBA')
    graphics[" "] = Image.new("RGBA", (16,16), (0,0,0,0))
    lvlImage = Image.new("RGBA", (width*tileSize, height*tileSize), (100,212,113,255))
    grid = Image.open("assets/grid.png").convert('RGBA')
    for y,r in enumerate(lvlRows):
        for x,c in enumerate(r):
            lvlImage.paste(grid, (x*tileSize, y*tileSize, (x+1)*tileSize, (y+1)*tileSize), grid)
            lvlImage.paste(graphics[c], (x*tileSize, y*tileSize, (x+1)*tileSize, (y+1)*tileSize), graphics[c])
    return lvlImage

def getNoisedState(numpyLevel, randomSize=20):
    stringLvl=numpyToString(numpyLevel)
    state=Sokoban.State()
    state.stringInitialize(stringLvl.split("\n"))
    for i in range(randomSize):
        if random.random() < 0.5:
            state.update(0, 2*random.randint(0,1) - 1)
        else:
            state.update(2*random.randint(0,1) - 1, 0)
    return stringToNumpy(str(state))

# Dataset Classes

In [2]:
import numpy as np
import torch
from torch.utils import data

class NormalDataset(data.Dataset):
    def __init__(self, file, start=-1, end=-1):
        data = np.load(file)["input"]
        np.random.shuffle(data)
        if start <= 0:
            start = 0
        if end <= 0:
            end = len(data)
        self.data = data[start:end].astype(np.float32)
        
    def __len__(self):
        return len(self.data)
        
    def __getitem__(self, idx):
        return torch.tensor(self.data[idx]), torch.tensor(self.data[idx])
    
class NoiseDataset(data.Dataset):
    def __init__(self, file, noiseLength=20, start=-1, end=-1):
        data = np.load(file)["input"]
        np.random.shuffle(data)
        if start <= 0:
            start = 0
        if end <= 0:
            end = len(data)
        self.data = data[start:end].astype(np.float32)
        self.noiseLength = noiseLength
        
    def __len__(self):
        return len(self.data)
        
    def __getitem__(self, idx):
        noiseData = getNoisedState(self.data[idx], self.noiseLength).astype(np.float32)
        return torch.tensor(noiseData), torch.tensor(self.data[idx])

# Pytorch Model

In [5]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Encoder(nn.Module):
    def __init__(self, encoding=8):
        super(Encoder, self).__init__()
        self.conv1 = nn.Conv2d(7, 128, kernel_size=(3,3), padding=2)
        self.conv2 = nn.Conv2d(128, 64, kernel_size=(3,3), padding=1)
        self.conv3 = nn.Conv2d(64, 32, kernel_size=(3,3), padding=1)
        self.fc1 = nn.Linear(32, encoding)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), (2,2)))
        x = F.relu(F.max_pool2d(self.conv2(x), (2,2)))
        x = F.relu(F.max_pool2d(self.conv3(x), (3,3)))
        x = x.view(-1, 32)
        x = torch.sigmoid(self.fc1(x))
        return x
    
class Decoder(nn.Module):
    def __init__(self, encoding=8):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(encoding, 32)
        self.conv1 = nn.Conv2d(32, 64, kernel_size=(3,3), padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=(3,3), padding=1)
        self.conv3 = nn.Conv2d(128, 7, kernel_size=(3,3), padding=0)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = x.view(-1, 32, 1, 1)
        x = F.relu(self.conv1(F.interpolate(x, scale_factor=3)))
        x = F.relu(self.conv2(F.interpolate(x, scale_factor=2)))
        x = F.softmax(self.conv3(F.interpolate(x, scale_factor=2)),dim=1)
        return x
    
class AutoEncoder(nn.Module):
    def __init__(self, encoding=8):
        super(AutoEncoder, self).__init__()
        self.encoder = Encoder(encoding)
        self.decoder = Decoder(encoding)
        
    def forward(self, x):
        self.bottleneck = self.encoder(x)
        return self.decoder(self.bottleneck)

# Training Code

In [6]:
import os
import matplotlib.pyplot as plt

def categorical_cross_entropy(y_pred, y_true):
    y_pred = torch.clamp(y_pred, 1e-7, 1 - 1e-7)
    return -(y_true * torch.log(y_pred)).sum(dim=1).mean()

def getHammingDistance(data1, data2):
    adata1=data1.argmax(axis=1)
    adata2=data2.argmax(axis=1)
    result=(adata1 != adata2).sum(dim=[1,2]).float()
    return result.mean()

def getPerfectRecostruction(data1, data2):
    adata1=data1.argmax(axis=1)
    adata2=data2.argmax(axis=1)
    result=(adata1-adata2).abs()
    result=1-result.clamp(0.0,1.0)
    result=result.sum(dim=[1,2]) == (data1.shape[2]*data1.shape[3])
    return result.sum()

def train(weightFolder, summaryWriter, logInterval, totalEpochs, network, device, trainLoader, valLoader):
    if weightFolder != None:
        os.makedirs(weightFolder)
    
    network.to(device)
    optimizer = optim.Adam(network.parameters())
    for epoch in range(totalEpochs):
        tLoss,tHamming,tCorrect,tBottleneck,tExamples = trainEpoch(network, device, optimizer, trainLoader)
        vLoss,vHamming,vCorrect,vBottleneck,vExamples = evalEpoch(network, device, valLoader)
        
        print('Epoch {}/{}:'.format(epoch+1, totalEpochs))
        print('Training:   Avg. Loss: {:.4f}, Avg. Hamming Distance: {:.4f}, Perfect Reconstruction: {}/{}'
              .format(tLoss, tHamming, tCorrect, len(trainLoader.dataset)))
        print('Validation: Avg. Loss: {:.4f}, Avg. Hamming Distance: {:.4f}, Perfect Reconstruction: {}/{}\n'
              .format(vLoss, vHamming, vCorrect, len(valLoader.dataset)))
        
        if epoch % logInterval == 0 or epoch == totalEpochs - 1:
            if summaryWriter != None:
                summaryWriter.add_scalars('Loss', {"train": tLoss, "val": vLoss}, epoch)
                summaryWriter.add_scalars('Hamming', {"train": tHamming / 100.0, "val": vHamming / 100.0}, epoch)
                summaryWriter.add_scalars('Bottleneck', {"train": tBottleneck, "val": vBottleneck}, epoch)

                trainLvls = [np.array(numpyToImage(lvl))/255.0 for lvl in tExamples[0:4]]
                summaryWriter.add_images('train-input', trainLvls, epoch, dataformats='HWC')
                trainLvls = [np.array(numpyToImage(lvl))/255.0 for lvl in tExamples[4:8]]
                summaryWriter.add_images('train-output', trainLvls, epoch, dataformats='HWC')
                valLvls = [np.array(numpyToImage(lvl))/255.0 for lvl in vExamples[0:4]]
                summaryWriter.add_images('val-input', valLvls, epoch, dataformats='HWC')
                valLvls = [np.array(numpyToImage(lvl))/255.0 for lvl in vExamples[4:8]]
                summaryWriter.add_images('val-output', valLvls, epoch, dataformats='HWC')

            if weightFolder != None:
                torch.save(network.state_dict(), weightFolder + 'model_{}_{:.4f}_{:.4f}.pth'.format(epoch+1,tLoss,vLoss))
                torch.save(optimizer.state_dict(), weightFolder + 'optimizer_{}_{:.4f}_{:.4f}.pth'.format(epoch+1,tLoss,vLoss))
    
def trainEpoch(network, device, optimizer, dataLoader):
    network.train()
    trainLoss = 0
    trainHamming = 0
    trainCorrect = 0
    trainBottleneck = 0
    trainExamples = []
    for batchID, (data, target) in enumerate(dataLoader):
        optimizer.zero_grad()
        data = data.to(device)
        target = target.to(device)
        output = network(data)
        loss = categorical_cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        
        trainLoss += loss.item()
        trainHamming += getHammingDistance(output, target)
        trainCorrect += getPerfectRecostruction(output, target)
        trainBottleneck += network.bottleneck.norm(dim=1).mean()
        
    for batchID, (data, target) in enumerate(dataLoader):
        data = data.to(device)
        target = target.to(device)
        output = network(data)
        trainExamples=torch.cat([target[0:4],output[0:4]], 0).cpu().detach().numpy()
        break
    
    trainLoss /= len(dataLoader)
    trainHamming /= len(dataLoader)
    trainBottleneck /= len(dataLoader)
    
    return trainLoss, trainHamming, trainCorrect, trainBottleneck, trainExamples
    
def evalEpoch(network, device, dataLoader):
    network.eval()
    testLoss = 0
    testHamming = 0
    testCorrect = 0
    testBottleneck = 0
    testExamples = []
    with torch.no_grad():
        for batchID, (data, target) in enumerate(dataLoader):
            data = data.to(device)
            target = target.to(device)
            output = network(data)
            testLoss += categorical_cross_entropy(output, target)
            testHamming += getHammingDistance(output, target)
            testCorrect += getPerfectRecostruction(output, target)
            testBottleneck += network.bottleneck.norm(dim=1).mean()
                
        for batchID, (data, target) in enumerate(dataLoader):
            data = data.to(device)
            target = target.to(device)
            output = network(data)
            testExamples=torch.cat([target[0:4],output[0:4]], 0).cpu().detach().numpy()
            break
        
        testLoss /= len(dataLoader)
        testHamming /= len(dataLoader)
        testBottleneck /= len(dataLoader)
    return testLoss, testHamming, testCorrect, testBottleneck, testExamples

# Testing Network

In [ ]:
from tensorboardX import SummaryWriter
import shutil

batchSize = 32
trainSet = 15000
noiseRollout = 50
totalEpochs = 2000
logInterval = 10
experimentName = "sigmoid_crossentropy_loss"
device = 'cuda' if torch.cuda.is_available() else 'cpu'

trainLoader = data.DataLoader(NormalDataset("FinalData/fixed_channel_first.npz", 0, trainSet),batch_size=batchSize, shuffle=True)
valLoader = data.DataLoader(NormalDataset("FinalData/fixed_channel_first.npz", trainSet),batch_size=batchSize, shuffle=True)

noiseTrainLoader = data.DataLoader(NoiseDataset("FinalData/fixed_channel_first.npz", noiseRollout, 0, trainSet),batch_size=batchSize, shuffle=True)
noiseValLoader = data.DataLoader(NoiseDataset("FinalData/fixed_channel_first.npz", noiseRollout, trainSet),batch_size=batchSize, shuffle=True)

trainers = {"Normal":(trainLoader, valLoader), "Noise":(noiseTrainLoader, noiseValLoader)}
encodingList = [32, 16, 8, 4]

if os.path.exists("Experiments/Weights/{}/".format(experimentName)):
    shutil.rmtree("Experiments/Weights/{}/".format(experimentName))
if os.path.exists("Experiments/Logs/{}/".format(experimentName)):
    shutil.rmtree("Experiments/Logs/{}/".format(experimentName))

for encoding in encodingList:
    for key, (tLoader, vLoader) in trainers.items():
        weightPath = "Experiments/Weights/{}/{}/Encoding_{}/".format(experimentName, key, encoding)
        logPath = "Experiments/Logs/{}/{}/Encoding_{}".format(experimentName, key, encoding)
        
        summaryWriter = SummaryWriter(logPath)
        autoencoder = AutoEncoder(encoding)
        train(weightPath, summaryWriter, logInterval, totalEpochs, autoencoder, device, tLoader, vLoader)

Epoch 1/2000:
Training:   Avg. Loss: 0.4818, Avg. Hamming Distance: 18.0332, Perfect Reconstruction: 0/15000
Validation: Avg. Loss: 0.3671, Avg. Hamming Distance: 14.5679, Perfect Reconstruction: 0/2584

Epoch 2/2000:
Training:   Avg. Loss: 0.3420, Avg. Hamming Distance: 13.4567, Perfect Reconstruction: 0/15000
Validation: Avg. Loss: 0.3254, Avg. Hamming Distance: 12.6646, Perfect Reconstruction: 0/2584

Epoch 3/2000:
Training:   Avg. Loss: 0.3156, Avg. Hamming Distance: 12.3323, Perfect Reconstruction: 0/15000
Validation: Avg. Loss: 0.3097, Avg. Hamming Distance: 12.1642, Perfect Reconstruction: 0/2584

Epoch 4/2000:
Training:   Avg. Loss: 0.3040, Avg. Hamming Distance: 11.9232, Perfect Reconstruction: 0/15000
Validation: Avg. Loss: 0.3001, Avg. Hamming Distance: 11.7449, Perfect Reconstruction: 3/2584

Epoch 5/2000:
Training:   Avg. Loss: 0.2939, Avg. Hamming Distance: 11.4988, Perfect Reconstruction: 2/15000
Validation: Avg. Loss: 0.2830, Avg. Hamming Distance: 10.8753, Perfect Reco

Epoch 41/2000:
Training:   Avg. Loss: 0.1698, Avg. Hamming Distance: 6.8270, Perfect Reconstruction: 267/15000
Validation: Avg. Loss: 0.1692, Avg. Hamming Distance: 6.7073, Perfect Reconstruction: 38/2584

Epoch 42/2000:
Training:   Avg. Loss: 0.1704, Avg. Hamming Distance: 6.8417, Perfect Reconstruction: 235/15000
Validation: Avg. Loss: 0.1689, Avg. Hamming Distance: 6.7591, Perfect Reconstruction: 53/2584

Epoch 43/2000:
Training:   Avg. Loss: 0.1703, Avg. Hamming Distance: 6.8344, Perfect Reconstruction: 219/15000
Validation: Avg. Loss: 0.1755, Avg. Hamming Distance: 7.0412, Perfect Reconstruction: 40/2584

Epoch 44/2000:
Training:   Avg. Loss: 0.1686, Avg. Hamming Distance: 6.7687, Perfect Reconstruction: 254/15000
Validation: Avg. Loss: 0.1745, Avg. Hamming Distance: 6.9877, Perfect Reconstruction: 42/2584

Epoch 45/2000:
Training:   Avg. Loss: 0.1660, Avg. Hamming Distance: 6.6702, Perfect Reconstruction: 259/15000
Validation: Avg. Loss: 0.1698, Avg. Hamming Distance: 6.7904, Per

Epoch 81/2000:
Training:   Avg. Loss: 0.1414, Avg. Hamming Distance: 5.6951, Perfect Reconstruction: 541/15000
Validation: Avg. Loss: 0.1559, Avg. Hamming Distance: 6.1433, Perfect Reconstruction: 84/2584

Epoch 82/2000:
Training:   Avg. Loss: 0.1436, Avg. Hamming Distance: 5.7744, Perfect Reconstruction: 511/15000
Validation: Avg. Loss: 0.1475, Avg. Hamming Distance: 5.8425, Perfect Reconstruction: 76/2584

Epoch 83/2000:
Training:   Avg. Loss: 0.1422, Avg. Hamming Distance: 5.7173, Perfect Reconstruction: 549/15000
Validation: Avg. Loss: 0.1431, Avg. Hamming Distance: 5.6929, Perfect Reconstruction: 89/2584

Epoch 84/2000:
Training:   Avg. Loss: 0.1408, Avg. Hamming Distance: 5.6561, Perfect Reconstruction: 568/15000
Validation: Avg. Loss: 0.1421, Avg. Hamming Distance: 5.6995, Perfect Reconstruction: 101/2584

Epoch 85/2000:
Training:   Avg. Loss: 0.1424, Avg. Hamming Distance: 5.7277, Perfect Reconstruction: 554/15000
Validation: Avg. Loss: 0.1398, Avg. Hamming Distance: 5.5550, Pe

Epoch 121/2000:
Training:   Avg. Loss: 0.1292, Avg. Hamming Distance: 5.2043, Perfect Reconstruction: 758/15000
Validation: Avg. Loss: 0.1387, Avg. Hamming Distance: 5.5201, Perfect Reconstruction: 76/2584

Epoch 122/2000:
Training:   Avg. Loss: 0.1284, Avg. Hamming Distance: 5.1622, Perfect Reconstruction: 741/15000
Validation: Avg. Loss: 0.1425, Avg. Hamming Distance: 5.6382, Perfect Reconstruction: 89/2584

Epoch 123/2000:
Training:   Avg. Loss: 0.1273, Avg. Hamming Distance: 5.1041, Perfect Reconstruction: 829/15000
Validation: Avg. Loss: 0.1326, Avg. Hamming Distance: 5.2433, Perfect Reconstruction: 126/2584

Epoch 124/2000:
Training:   Avg. Loss: 0.1272, Avg. Hamming Distance: 5.0990, Perfect Reconstruction: 788/15000
Validation: Avg. Loss: 0.1270, Avg. Hamming Distance: 5.0145, Perfect Reconstruction: 141/2584

Epoch 125/2000:
Training:   Avg. Loss: 0.1247, Avg. Hamming Distance: 5.0295, Perfect Reconstruction: 883/15000
Validation: Avg. Loss: 0.1333, Avg. Hamming Distance: 5.32

Epoch 161/2000:
Training:   Avg. Loss: 0.1188, Avg. Hamming Distance: 4.7590, Perfect Reconstruction: 991/15000
Validation: Avg. Loss: 0.1270, Avg. Hamming Distance: 5.0028, Perfect Reconstruction: 148/2584

Epoch 162/2000:
Training:   Avg. Loss: 0.1160, Avg. Hamming Distance: 4.6677, Perfect Reconstruction: 1061/15000
Validation: Avg. Loss: 0.1238, Avg. Hamming Distance: 4.8182, Perfect Reconstruction: 164/2584

Epoch 163/2000:
Training:   Avg. Loss: 0.1150, Avg. Hamming Distance: 4.6286, Perfect Reconstruction: 1156/15000
Validation: Avg. Loss: 0.1223, Avg. Hamming Distance: 4.8147, Perfect Reconstruction: 180/2584

Epoch 164/2000:
Training:   Avg. Loss: 0.1166, Avg. Hamming Distance: 4.6797, Perfect Reconstruction: 1045/15000
Validation: Avg. Loss: 0.1314, Avg. Hamming Distance: 5.2014, Perfect Reconstruction: 101/2584

Epoch 165/2000:
Training:   Avg. Loss: 0.1180, Avg. Hamming Distance: 4.7389, Perfect Reconstruction: 1046/15000
Validation: Avg. Loss: 0.1225, Avg. Hamming Distance

...

Epoch 421/2000:
Training:   Avg. Loss: 0.1784, Avg. Hamming Distance: 6.9400, Perfect Reconstruction: 362/15000
Validation: Avg. Loss: 0.1810, Avg. Hamming Distance: 6.9320, Perfect Reconstruction: 77/2584

Epoch 422/2000:
Training:   Avg. Loss: 0.1756, Avg. Hamming Distance: 6.8289, Perfect Reconstruction: 425/15000
Validation: Avg. Loss: 0.1825, Avg. Hamming Distance: 7.0198, Perfect Reconstruction: 81/2584

Epoch 423/2000:
Training:   Avg. Loss: 0.1779, Avg. Hamming Distance: 6.9174, Perfect Reconstruction: 391/15000
Validation: Avg. Loss: 0.1840, Avg. Hamming Distance: 7.1083, Perfect Reconstruction: 72/2584

Epoch 424/2000:
Training:   Avg. Loss: 0.1757, Avg. Hamming Distance: 6.8350, Perfect Reconstruction: 381/15000
Validation: Avg. Loss: 0.1822, Avg. Hamming Distance: 6.9898, Perfect Reconstruction: 70/2584

Epoch 425/2000:
Training:   Avg. Loss: 0.1765, Avg. Hamming Distance: 6.8488, Perfect Reconstruction: 399/15000
Validation: Avg. Loss: 0.1962, Avg. Hamming Distance: 7.4069

Epoch 461/2000:
Training:   Avg. Loss: 0.1763, Avg. Hamming Distance: 6.8410, Perfect Reconstruction: 407/15000
Validation: Avg. Loss: 0.1774, Avg. Hamming Distance: 6.8053, Perfect Reconstruction: 80/2584

Epoch 462/2000:
Training:   Avg. Loss: 0.1737, Avg. Hamming Distance: 6.7388, Perfect Reconstruction: 432/15000
Validation: Avg. Loss: 0.1963, Avg. Hamming Distance: 7.3400, Perfect Reconstruction: 73/2584

Epoch 463/2000:
Training:   Avg. Loss: 0.1760, Avg. Hamming Distance: 6.8295, Perfect Reconstruction: 388/15000
Validation: Avg. Loss: 0.1800, Avg. Hamming Distance: 6.8477, Perfect Reconstruction: 82/2584

Epoch 464/2000:
Training:   Avg. Loss: 0.1749, Avg. Hamming Distance: 6.7815, Perfect Reconstruction: 425/15000
Validation: Avg. Loss: 0.1867, Avg. Hamming Distance: 7.1669, Perfect Reconstruction: 56/2584

Epoch 465/2000:
Training:   Avg. Loss: 0.1731, Avg. Hamming Distance: 6.7223, Perfect Reconstruction: 441/15000
Validation: Avg. Loss: 0.1813, Avg. Hamming Distance: 6.9181

Epoch 501/2000:
Training:   Avg. Loss: 0.1729, Avg. Hamming Distance: 6.6996, Perfect Reconstruction: 447/15000
Validation: Avg. Loss: 0.1829, Avg. Hamming Distance: 6.9703, Perfect Reconstruction: 72/2584

Epoch 502/2000:
Training:   Avg. Loss: 0.1736, Avg. Hamming Distance: 6.7316, Perfect Reconstruction: 426/15000
Validation: Avg. Loss: 0.1832, Avg. Hamming Distance: 6.9865, Perfect Reconstruction: 73/2584

Epoch 503/2000:
Training:   Avg. Loss: 0.1748, Avg. Hamming Distance: 6.7811, Perfect Reconstruction: 388/15000
Validation: Avg. Loss: 0.1803, Avg. Hamming Distance: 6.9082, Perfect Reconstruction: 73/2584

Epoch 504/2000:
Training:   Avg. Loss: 0.1730, Avg. Hamming Distance: 6.7166, Perfect Reconstruction: 431/15000
Validation: Avg. Loss: 0.1805, Avg. Hamming Distance: 6.9183, Perfect Reconstruction: 86/2584

Epoch 505/2000:
Training:   Avg. Loss: 0.1749, Avg. Hamming Distance: 6.7968, Perfect Reconstruction: 410/15000
Validation: Avg. Loss: 0.1861, Avg. Hamming Distance: 7.1310

Epoch 541/2000:
Training:   Avg. Loss: 0.1715, Avg. Hamming Distance: 6.6429, Perfect Reconstruction: 437/15000
Validation: Avg. Loss: 0.1764, Avg. Hamming Distance: 6.7009, Perfect Reconstruction: 91/2584

Epoch 542/2000:
Training:   Avg. Loss: 0.1732, Avg. Hamming Distance: 6.7214, Perfect Reconstruction: 414/15000
Validation: Avg. Loss: 0.1828, Avg. Hamming Distance: 6.9492, Perfect Reconstruction: 69/2584

Epoch 543/2000:
Training:   Avg. Loss: 0.1720, Avg. Hamming Distance: 6.6596, Perfect Reconstruction: 450/15000
Validation: Avg. Loss: 0.1802, Avg. Hamming Distance: 6.8776, Perfect Reconstruction: 79/2584

Epoch 544/2000:
Training:   Avg. Loss: 0.1718, Avg. Hamming Distance: 6.6572, Perfect Reconstruction: 446/15000
Validation: Avg. Loss: 0.1763, Avg. Hamming Distance: 6.7333, Perfect Reconstruction: 91/2584

Epoch 545/2000:
Training:   Avg. Loss: 0.1703, Avg. Hamming Distance: 6.6124, Perfect Reconstruction: 465/15000
Validation: Avg. Loss: 0.1859, Avg. Hamming Distance: 7.0538

Epoch 581/2000:
Training:   Avg. Loss: 0.1706, Avg. Hamming Distance: 6.6112, Perfect Reconstruction: 455/15000
Validation: Avg. Loss: 0.1755, Avg. Hamming Distance: 6.7171, Perfect Reconstruction: 97/2584

Epoch 582/2000:
Training:   Avg. Loss: 0.1730, Avg. Hamming Distance: 6.6841, Perfect Reconstruction: 417/15000
Validation: Avg. Loss: 0.1825, Avg. Hamming Distance: 6.9439, Perfect Reconstruction: 84/2584

Epoch 583/2000:
Training:   Avg. Loss: 0.1690, Avg. Hamming Distance: 6.5451, Perfect Reconstruction: 465/15000
Validation: Avg. Loss: 0.1748, Avg. Hamming Distance: 6.6701, Perfect Reconstruction: 94/2584

Epoch 584/2000:
Training:   Avg. Loss: 0.1708, Avg. Hamming Distance: 6.6167, Perfect Reconstruction: 427/15000
Validation: Avg. Loss: 0.1800, Avg. Hamming Distance: 6.8476, Perfect Reconstruction: 83/2584

Epoch 585/2000:
Training:   Avg. Loss: 0.1711, Avg. Hamming Distance: 6.6249, Perfect Reconstruction: 409/15000
Validation: Avg. Loss: 0.1763, Avg. Hamming Distance: 6.6682

# Data Testing

In [ ]:
import matplotlib.pyplot as plt

noiseTrainLoader = data.DataLoader(NoiseDataset("FinalData/fixed_channel_first.npz", 50, 0, 15000),batch_size=128, shuffle=True)

for idx, (miniInput,miniOutput) in enumerate(noiseTrainLoader):
    if idx == 0:
        result = torch.cat([miniInput[0:4],miniOutput[0:4]],0)
        plt.figure()
        plt.imshow(numpyToImage(result[0]))
        plt.show()
        
        plt.figure()
        plt.imshow(numpyToImage(result[4]))
        plt.show()

# Fix the Dataset

In [ ]:
levels = np.load('FinalData/channel_first_data.npz')

directions = {(-1,0), (1,0), (0,1), (0,-1)}

def floodFill(lvl, x, y, label):
    result = []
    queue = [(x,y)]
    while(len(queue) > 0):
        current = queue.pop(0)
        if lvl[current[1]][current[0]] == 0:
            lvl[current[1]][current[0]] = label
            result.append(current)
            for d in directions:
                new = (current[0]+d[0], current[1]+d[1])
                if new[0] < 0 or new[1] < 0 or new[0] >= lvl.shape[1] or new[1] >= lvl.shape[0]:
                    continue
                queue.append(new)
    return result

def floodFillCheck(lvl):
    finalResult=[]
    tempLvl = np.zeros((10,10))
    for y in range(lvl.shape[0]):
        for x in range(lvl.shape[1]):
            if lvl[y][x] == 0:
                tempLvl[y][x] = -1
    for y in range(lvl.shape[0]):
        for x in range(lvl.shape[1]):
            if tempLvl[y][x] == 0:
                finalResult.append(floodFill(tempLvl,x,y,len(finalResult)+1))
    return finalResult

def fixLevel(lvl, regions):
    fixed = np.zeros(lvl.shape)
    for y in range(lvl.shape[0]):
        for x in range(lvl.shape[1]):
            fixed[y][x] = lvl[y][x]
    maxRegion = []
    for r in regions:
        if len(r) > len(maxRegion):
            maxRegion = r
    for r in regions:
        if r != maxRegion:
            for p in r:
                fixed[p[1]][p[0]] = 0
    finalLvl = np.zeros((7,10,10))
    for y in range(fixed.shape[0]):
        for x in range(fixed.shape[1]):
            finalLvl[int(fixed[y][x])][y][x] = 1
    return finalLvl

fixedLevels = []
total = 0
for lvl in levels["input"]:
    total += 1
    regions = floodFillCheck(np.argmax(lvl,axis=0))
    if len(regions) > 1:
        fixedLevels.append(fixLevel(np.argmax(lvl,axis=0), regions))
    else:
        fixedLevels.append(lvl)

np.savez('FinalData/fixed_channel_first.npz', input=fixedLevels, output=fixedLevels)